```python
tf.one_hot(indices, depth, on_value=None, off_value=None, axis=None, dtype=None, name=None)

```

-   `indices`: 整数张量，包含了要转换为 one-hot 编码的索引值。
-   `depth`: 表示 one-hot 向量的长度，即总类别的数量。
-   `on_value` 和 `off_value`: 分别指定 one-hot 向量中激活位（对应索引）和非激活位的值，默认为1和0。
-   `axis`: 指定 one-hot 向量应该添加在哪一轴上，默认情况下将其添加为新的最末尾轴。
-   `dtype`: 指定输出张量的数据类型，默认为 `tf.float32`。
-   `name`: 可选的命名空间名称，用于区分不同的运算。


In [1]:
# pip install pandas matplotlib scikit-learn-intelex scikit-learn openpyxl tensorboard seaborn

from numpy import disp
import tensorflow as tf  # version : '1.12.0'

NUM_CLASSES = 10 # 10分类
labels = [0,1,2,3] # sample label
batch_size = tf.size(labels) # get size of labels : 4
labels = tf.expand_dims(labels, 1) # 增加一个维度
indices = tf.expand_dims(tf.range(0, batch_size,1), 1) #生成索引
concated = tf.concat([indices, labels] , 1) #作为拼接
# onehot_labels = tf.one_hot(concated, tf.stack([batch_size, NUM_CLASSES]), 1.0, 0.0) # 生成one-hot编码的标签

# 使用tf.one_hot直接将稀疏标签转换为one-hot编码
onehot_labels = tf.one_hot(labels, depth=NUM_CLASSES)

display(onehot_labels)

2024-03-11 19:51:40.926471: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 19:51:40.946512: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 19:51:40.946535: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 19:51:40.947007: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 19:51:40.950262: I tensorflow/core/platform/cpu_feature_guar

<tf.Tensor: shape=(4, 1, 10), dtype=float32, numpy=
array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]], dtype=float32)>

In [2]:
import numpy as np
import tensorflow as tf

# 得到4个5维独热行向量向量,
#    其中第1个向量的第0个分量是独热1，
#    第2个向量的第2个分量是独热，
#    第3个向量没有独热，因为指定为-1
#    第4个向量的第1个分量为独热
# labels向targets的转变
labels = [0, 2, -1, 1]
# labels是shape=(4,)的张量。则返回的targets是shape=(len(labels), depth)张量。
# 且这种情况下,axis=-1等价于axis=1
targets = tf.one_hot(indices=labels, depth=5, on_value=1.0, off_value=0.0, axis=-1)

print(targets)


# 得到1个5维独热行向量。
targets = tf.one_hot(indices=3, depth=5, on_value=1.0, off_value=0.0, axis=0)

print(targets)

# 得到1个5维独热列向量
targets = tf.one_hot(indices=[3], depth=5, on_value=1.0, off_value=0.0, axis=0)

print(targets)

targets = tf.one_hot(indices=[[0,1],[1,0]], depth=3)

print(targets)
 
# 确保 labels 是一个 NumPy 数组或者类似的数据结构
labels = np.array([0, 1, 2, 3])

# 在 TensorFlow 2.x 中，不需要使用 session，可以直接运行
targets = tf.one_hot(indices=labels, depth=5, on_value=1.0, off_value=0.0, axis=-1)
print(targets)


tf.Tensor(
[[1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]], shape=(4, 5), dtype=float32)
tf.Tensor([0. 0. 0. 1. 0.], shape=(5,), dtype=float32)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [1.]
 [0.]], shape=(5, 1), dtype=float32)
tf.Tensor(
[[[1. 0. 0.]
  [0. 1. 0.]]

 [[0. 1. 0.]
  [1. 0. 0.]]], shape=(2, 2, 3), dtype=float32)
tf.Tensor(
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]], shape=(4, 5), dtype=float32)


这个问题通常是由于在使用自定义的损失函数时，将one-hot编码的标签张量（y_true）直接传递给损失函数而导致的。在这种情况下，直接对one-hot编码的张量进行操作可能会导致梯度消失，因为这些张量包含很多零值。为了避免这个问题，我们可以使用Tensorflow内置的函数来转换标签张量为稠密张量而不是one-hot编码的张量。

以下是一个示例的解决方法，其中将使用SparseCategoricalCrossentropy作为示例损失函数：

import tensorflow as tf

def my_loss(y_true, y_pred):
    y_true = tf.squeeze(y_true) # 去掉size为1的维度
    loss = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred)
    return loss

model.compile(optimizer='adam', loss=my_loss)
在这个示例中，我们使用了Tensorflow的squeeze函数来去掉标签张量中size为1的维度，然后将其传递给SparseCategoricalCrossentropy函数。这将创建一个稠密的标签张量，从而避免了梯度消失的问题。

In [3]:
import tensorflow as tf

def my_loss(y_true, y_pred):
    y_true = tf.squeeze(y_true) # 去掉size为1的维度
    loss = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred)
    return loss

model.compile(optimizer='adam', loss=my_loss)

NameError: name 'model' is not defined